In [71]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.spatial.distance as dist
%matplotlib inline
import pylab
import json
import random
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error as mse

from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#df_users = pd.read_json('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/user.json', lines=True)

In [4]:
#df_users.head()

# Reading big file line by line, taking first 10000 reviews

In [34]:
c=0
s = open('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/review_sub.json','w',encoding='utf8')
with open('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/review.json', encoding='utf8') as f:
    for line in f:
        c=c+1
        s.write(line)
        if(c == 10000):
            break

In [35]:
df_reviews = pd.read_json('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/review_sub.json', lines=True)

In [36]:
df_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw


# Read Business Data Set

In [37]:
df_bus = pd.read_json('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/business.json', lines=True)

# Merge Business and Review dataset

In [38]:
res_mer = pd.merge(df_bus,df_reviews, on='business_id')

In [39]:
res_mer.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,stars_x,state,cool,date,funny,review_id,stars_y,text,useful,user_id
0,350 E Steels Corners Rd,"{'DogsAllowed': True, 'OutdoorSeating': True, ...",PjQngP_7m7PA8K1WUHdXqg,"[Restaurants, American (New), Hot Dogs, Burgers]",Cuyahoga Falls,"{'Friday': '11:00-22:00', 'Tuesday': '11:00-21...",1,41.180715,-81.494772,RETRO Dog,...,3.5,OH,0,2013-08-30,0,yopVrikncv_54eR44R9sHQ,4,Great variety and the best dog in town. Try th...,0,Q9nZLjnQ_0RiWfNjCRt9sQ
1,350 E Steels Corners Rd,"{'DogsAllowed': True, 'OutdoorSeating': True, ...",PjQngP_7m7PA8K1WUHdXqg,"[Restaurants, American (New), Hot Dogs, Burgers]",Cuyahoga Falls,"{'Friday': '11:00-22:00', 'Tuesday': '11:00-21...",1,41.180715,-81.494772,RETRO Dog,...,3.5,OH,0,2013-12-09,0,fQn5gA-jXYHefi2Kxx9rcg,1,This was my first visit and I was definitely N...,1,9Vu2eIk0HXGAD2h1Vr81RA
2,350 E Steels Corners Rd,"{'DogsAllowed': True, 'OutdoorSeating': True, ...",PjQngP_7m7PA8K1WUHdXqg,"[Restaurants, American (New), Hot Dogs, Burgers]",Cuyahoga Falls,"{'Friday': '11:00-22:00', 'Tuesday': '11:00-21...",1,41.180715,-81.494772,RETRO Dog,...,3.5,OH,3,2012-06-19,3,HzreYQD3_VOa-MvleCQhEw,4,Went to their grand opening on the June 17th a...,6,MBf09zdFcOEnYA84aENzXA
3,350 E Steels Corners Rd,"{'DogsAllowed': True, 'OutdoorSeating': True, ...",PjQngP_7m7PA8K1WUHdXqg,"[Restaurants, American (New), Hot Dogs, Burgers]",Cuyahoga Falls,"{'Friday': '11:00-22:00', 'Tuesday': '11:00-21...",1,41.180715,-81.494772,RETRO Dog,...,3.5,OH,0,2015-10-27,0,E73McNvR0dT-l-7mlsh_7A,3,Good food. Way over priced as everything is ex...,2,cA7hAhlsXIotA4wRLdAZBQ
4,350 E Steels Corners Rd,"{'DogsAllowed': True, 'OutdoorSeating': True, ...",PjQngP_7m7PA8K1WUHdXqg,"[Restaurants, American (New), Hot Dogs, Burgers]",Cuyahoga Falls,"{'Friday': '11:00-22:00', 'Tuesday': '11:00-21...",1,41.180715,-81.494772,RETRO Dog,...,3.5,OH,0,2015-11-18,0,6D0JvHSBurLYbC_o_drcJA,4,This is my regular lunch stop when I am in Sto...,0,UCnk2nOq6cvtjboMWkhO9A


# Utility Matrix Creation

In [40]:
df = res_mer.pivot(index = 'user_id', columns ='business_id', values='stars_y')
df.head()

business_id,-MhfebM0QIsKt87iDN-FNw,-cYOKJ5kbVZqzSYQIzZcqA,-iC6giVhI7vY6W_dnw08YA,-pV9kWNoA9vyHfM_auYecA,03SYJLErY8XpNfY-qiDZcw,0nyM_mub7LxN6Dx7tYpMaA,1pXyaPUXMqZyxW1Ew7BLUg,2L4rNP5kjRXCxxspYPRTdA,2UmvDC88Wj3eKOBLFTh86w,2Z2QuLYY0FvUeJ75ane-Zw,...,wzuhuTptNFTi4MZkrCItBA,xOx-N9G7AteZTnZfVFfNHA,xjViuMrrApPM6Rnz8ovOVw,xpW7Qp8JpYys0IGG0ENZaw,y7yo-LAjJnNoYwWX6ZYbqA,yRk8ezAFV59HDHsuHiiU6Q,ykSlMTRgyWzAmhP7ywc9gg,ypuo8mzg0uCxl24K08rIig,zoq5SERVEDBd7MaKPjlLJA,zw4Legbcu018p5WcZ74iWA
user_id,,,,,,,,,,,,,,,,,,,,,
---1lKK3aKOuomHnwAkAow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--Nnm_506G_p8MxAOQna5w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--P-Qvza7AED8gnDrZkMgA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ZNfWKj1VyVElRx6-g1fg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-00MbjbaOISrcuV7jOVRIg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
items=df.columns.values

In [42]:
mat = df.as_matrix().astype(np.float)

# Centralizing each row vector to mean zero

In [43]:
sum =0.0
count =0
for i in range(len(mat)):
    sum = 0.0
    count =0
    for j in range(len(mat[0])):
        if(math.isnan(mat[i][j])):
            continue
        sum += mat[i][j]
        count += 1
    avg = sum/count
    for j in range(len(mat[0])):
        if(math.isnan(mat[i][j])):
            continue
        mat[i][j] -= avg

In [44]:
# Replace NaN values with zero in matrix
ind = np.where(np.isnan(mat))
mat[ind] = 0

In [45]:
mat

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [68]:
print("Number of users:", len(mat))
print("Number of items:", len(mat[0]))

Number of users: 9585
Number of items: 227


# Similarity Matrix

In [47]:
def sim_matrix(u, eps=1.0e-9):
    step1 = u.dot(u.T) + eps
    step2 = np.array([np.sqrt(np.diagonal(step1))])
    return (step1 / step2 / step2.T)

# Top K Similar Items

In [48]:
def top_k(arr, self_idx, k):
    sortedArr=arr.argsort()[::-1][:len(arr)]
    sortedArr=sortedArr[1:k+1]
    tk={}
    for i in sortedArr:
        if i != self_idx:
            tk[i]=arr[i]
    return tk

# Recommendation via Similar Users

In [49]:
def rec_via_users(m_utility, m_sim_users, user_idx, item_idx, k):
    dict_s = top_k(m_sim_users[user_idx], user_idx, k)
    s = 0.00
    c = 0.00
    for k,v in dict_s.items():
        if m_utility[k][item_idx] == 0:
            continue
        s += (m_utility[k][item_idx] * v)
        c += v
    if c == 0:
        return 0
    return s/c

In [50]:
# Finding user-index with atleast 5 ratings for items
i = 0
for i in range(len(mat)):
    c = 0
    for j in range(len(mat[0])):
        if mat[i][j] > 0:
            c = c+1
    if c >4:
        break
print(i)

3939


In [58]:
#u1 = np.array([[2, 0, 0, 4, 4], [5, 5, 5, 3, 3], [2, 4, 2, 1, 2]])
#Expected Rating of item number 4 for user number 3939
print(rec_via_users(mat, sim_matrix(mat), 3939, 4, 10))

0


# SVD Recommendation

In [74]:
random.seed(12345)

def mse_utility(u1, u2):
    return mse(u1[u1.nonzero()].flatten(), u2[u2.nonzero()].flatten())

def svd_projection(m_utility, num_dims):
    U,S,VT = svds(m_utility, num_dims)
    SD = np.diag(S)
    return np.dot(np.dot(U,SD),VT)

def recs_via_svd(m_utility, num_dims, test_n):
    test = random.sample(range(m_utility.shape[0]), test_n)    
    svd_utility = svd_projection(m_utility, num_dims)
    
    true = []
    pred = []
    for user_idx in test:
        for item_idx in range(m_utility.shape[1]):
            if m_utility[user_idx][item_idx] != 0:
                true.append(m_utility[user_idx][item_idx])
                p = round(svd_utility[user_idx][item_idx])
                
                if p != 0:
                    pred.append(p)
                else:
                    pred.append(1.0e-9)

    return mse_utility(np.array([true], dtype=np.float64), np.array([pred], dtype=np.float64))



# Mean Square Error for rating prediction by SVD
print(recs_via_svd(mat, 200, 9500))

0.160562208905


# End of SVD #

In [34]:
# List of indices of similar users
similar_users = []
similarity = []
def cosine_similarity(user):
    for i in range(len(mat)):
        cs = 1 - dist.cosine(mat[i],user)
        if cs > 0:
            similarity.append(cs)
            similar_users.append(i)

In [35]:
#calling method
#Let new user be mat[320]
cosine_similarity(mat[320])

/home/gautam/.local/lib/python3.5/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [27]:
len(similar_users)

200

In [47]:
##Item Rating prediction for user
###INCORRECT - need to check the formula
item_ratings = np.zeros((len(mat[0])))
for i in range(len(similar_users)):
    u = mat[similar_users[i]]
    for j in range(len(u)):
        if(math.isnan(u[j])):
            continue
        else:
            item_ratings[j] += similarity[i] * u[j]

In [49]:
##Items recommended to user based on rating prediction
c = 0
for i in range(len(item_ratings)):
    if item_ratings[i]>0:
        c = c +1
print("count is:", c)

count is: 131


In [50]:
###Item Recommendation to users based on similar_users####
ind = similar_users[1]
recommend_items_indices = []

for i in range(len(similar_users)):
    for k,v in enumerate(mat[similar_users[i]]):
        if v>0:
            recommend_items_indices.append(k)

##Removing indices which are already used by user
for k,v in enumerate(mat[320]):
    if v > 0:
        recommend_items_indices.remove(k)
            
#Recommended items is
recommended_items = []

for i in range(len(recommend_items_indices)):
    recommended_items.append(items[recommend_items_indices[i]])


In [51]:
## recommended_items - user_items
len(recommended_items)

531

In [30]:
df_bus[df_bus['business_id'] == recommended_items[0]]

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
33553,"3015 S Valley View Blvd, Ste 100",{'BusinessAcceptsCreditCards': True},MPGrHMnDFzXCr-EtNSDh2g,"[Automotive, Oil Change Stations, Smog Check S...",Las Vegas,"{'Monday': '7:30-17:00', 'Thursday': '7:30-17:...",1,36.136304,-115.191512,Bob's Autodynamics,Westside,89102,15,4.0,NV
